In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import Dataset

In [3]:
import sys
sys.path.append("../src")
sys.path.append("../../pyASBC/src")
sys.path.append("../../CompoTree/src")

In [4]:
from fluidvec import *
import pyASBC
from CompoTree import ComponentTree

In [5]:
asbc = pyASBC.Asbc5Corpus()
ctree = ComponentTree.load()

In [6]:
from itertools import islice
sent_iter = asbc.iter_sentences()
sents = list(islice(sent_iter, 0, 10))

In [7]:
word = sents[0][0]

In [8]:
cc = ctree.query("時", use_flag="shortest", max_depth=1)[0]

In [9]:
cc.components()

['日', '寺']

In [10]:
word_vocab = Vocabulary()
char_vocab = Vocabulary()
compo_vocab = Vocabulary()

In [11]:
word_vocab.add("測試")
word_vocab.add("測試")
word_vocab.add("程式")
assert word_vocab.encode("測試") == 2
assert word_vocab.decode(2) == "測試"
assert len(word_vocab) == 4

In [12]:
import pickle
from tqdm.auto import tqdm
word_iter = asbc.iter_words()
# word_iter = islice(word_iter, 0, 100)
for word in tqdm(word_iter):
    components, chars, word = make_word_tuple(word, ctree)
    for compo_x in components:
        compo_vocab.add(compo_x)
    for char_x in chars:
        char_vocab.add(char_x)
    word_vocab.add(word)
compo_vocab.save("../data/compo_vocab_test.pkl")
char_vocab.save("../data/char_vocab_test.pkl")
word_vocab.save("../data/word_vocab_test.pkl")

len(compo_vocab), len(char_vocab), len(word_vocab)

(3504, 20396, 217350)

In [13]:
compo_vocab = Vocabulary.load("../data/compo_vocab.pkl")
char_vocab = Vocabulary.load("../data/char_vocab.pkl")
word_vocab = Vocabulary.load("../data/word_vocab.pkl")
len(compo_vocab), len(char_vocab), len(word_vocab)

(3289, 20396, 217350)